In [1]:
import pandas as pd
import numpy as np
import sklearn.ensemble
import itertools
import matplotlib.pyplot as plt

In [2]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn import svm

In [3]:
#reading the data file
raw_file = pd.read_csv('no_NAs_A.raw', sep=" ")
raw_file.head(3)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,rs12565286_C,rs12082473_A,rs3094315_G,rs2286139_T,...,rs5770820_A,rs9628187_T,rs6010063_A,rs715586_T,rs8137951_A,rs3810648_G,rs3865766_C,rs2238837_C,rs34726907_T,rs28729663_A
0,NA06984,NA06984,0,0,0,-9,0,0,0,2,...,0,0,2,0,0,0,2,0,0,0
1,NA06985,NA06985,0,0,0,-9,0,0,0,2,...,1,0,0,0,2,1,0,1,1,1
2,NA06986,NA06986,0,0,0,-9,0,0,0,2,...,0,1,1,0,1,0,2,0,0,0


In [4]:
#deleting the columns not part of the model
raw_file = raw_file.drop(raw_file.columns[[0,1,2,3,4,5]],axis=1)

In [5]:
snps_pyth = list(raw_file.columns.values)

In [6]:
len(snps_pyth)

857124

In [15]:
pops=pd.read_csv("C://Users//WATS5I//prjct_SVM//pop_names", sep=" ",nrows=268)
pops=pops.drop('IID',axis=1)

In [18]:
y = np.array(pops['pop'])
#converting the covariates into a matrix
X_train, X_test, ytrain, ytest = train_test_split(raw_file,y,test_size=0.20)
X = raw_file.as_matrix(columns=None)

In [19]:
##support vector machine 
linear_svc= svm.SVC(kernel='linear',C=1).fit(X_train,ytrain)
# rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=1).fit(X_train, ytrain # disabled 

In [31]:
suprtvc_1=linear_svc.coef_[0]

In [32]:
len(suprtvc_1)

857124

In [46]:
#this method loops through each SVC and gets the list of snps having the coefficient higher than the threshold
def getSVC_snps(threshold):
    consolidated_snps = {} #consolidated list of SNPs 
    for i in range(0,2):#looping through 3 populations and extracting coeff.
        suprtvc = linear_svc.coef_[i]
        suprtvc_abs = abs(suprtvc)
        num_snps = len(snps_pyth)
        snp_dict = {} #dictionary key:->snp and value:->coefficient
        for j in range(0,num_snps):
            snp_dict[snps_pyth[j]] = suprtvc_abs[j]
        top_snps,top_values = [],[]
        snp_dict_sorted = sorted(snp_dict, key=snp_dict.get, reverse=True)#sorting the dictionary
        for snp in snp_dict_sorted[:threshold]:
            coeff = snp_dict[snp]
            if snp in consolidated_snps:
                if(coeff>consolidated_snps[snp]):
                    consolidated_snps[snp] = coeff
            else:
                consolidated_snps[snp] = coeff
    return consolidated_snps

In [47]:
consolidated_snps = getSVC_snps(500)

In [50]:
sorted_consolidated_snps = sorted(consolidated_snps, key=consolidated_snps.get, reverse=True)#sorting the dictionary
top_100_consolidated = sorted_consolidated_snps[:100]

In [51]:
fw = open("top500_snps.txt","w")
fw.write("SNP\tScore\n")
for snp in top_100_consolidated:
    fw.write(snp+"\t")
    fw.write(str(consolidated_snps[snp]))
    fw.write('\n')
fw.close()

In [48]:
consolidated_snps

{'rs10005626_A': 3.1158090115032328e-05,
 'rs10007705_T': 3.1158090115032328e-05,
 'rs10036748_C': 2.9815510392100232e-05,
 'rs10041072_C': 3.0828552603971388e-05,
 'rs10044496_T': 7.6180674218073277e-05,
 'rs10048370_C': 3.1106175751586864e-05,
 'rs10076497_C': 7.4088484185404796e-05,
 'rs10076782_G': 7.7457017042407538e-05,
 'rs10114731_C': 7.061839476339139e-05,
 'rs1012023_C': 3.126804374593698e-05,
 'rs10120419_C': 7.1313569559223731e-05,
 'rs1012189_A': 2.9324815459568464e-05,
 'rs10123583_A': 7.0141031032309567e-05,
 'rs10129619_G': 9.1110246034326457e-05,
 'rs10133798_T': 7.0913907410845116e-05,
 'rs10137275_T': 7.4345701198957724e-05,
 'rs10142116_A': 3.1626707684584617e-05,
 'rs1014513_G': 3.0027058862204254e-05,
 'rs1015218_C': 9.2647475668607092e-05,
 'rs10153988_T': 7.990818464595248e-05,
 'rs10156623_C': 3.1996916710036951e-05,
 'rs10164529_C': 7.0609311957075949e-05,
 'rs10170704_C': 2.9148076952146128e-05,
 'rs10186877_C': 3.0120733503097758e-05,
 'rs10197003_T': 7.2882

In [52]:
linear_svc.n_support_

array([54, 74, 86])

In [53]:
linear_svc.dual_coef_

array([[  4.60072892e-07,   0.00000000e+00,   2.65961330e-07,
          9.68387766e-07,   0.00000000e+00,   6.59041920e-07,
          0.00000000e+00,   1.48500523e-06,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.82026629e-06,
          0.00000000e+00,   1.58435114e-06,   7.50792308e-07,
          0.00000000e+00,   0.00000000e+00,   8.33979019e-07,
          0.00000000e+00,   3.36816568e-07,   0.00000000e+00,
          1.83098018e-06,   3.90341664e-07,   7.65718999e-07,
          0.00000000e+00,   3.77509675e-07,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.05587722e-06,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          2.91559625e-07,   0.00000000e+00,   0.00000000e+00,
          1.27832050e-06,   0.00000000e+00,   0.00000000e+00,
          7.74934191e-07,   0.00000000e+00,   0.00000000e+00,
          2.75038241e-06,   9.46002963e-07,   5.15266330e-06,
        

In [55]:
linear_svc.support_vectors_

array([[ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  2., ...,  2.,  0.,  1.],
       [ 0.,  0.,  2., ...,  0.,  1.,  1.],
       ..., 
       [ 0.,  0.,  2., ...,  1.,  1.,  1.],
       [ 0.,  0.,  2., ...,  0.,  0.,  0.],
       [ 0.,  0.,  2., ...,  2.,  2.,  2.]])

In [56]:
Svectors=linear_svc.support_vectors_

In [57]:
Svectors.shape

(214, 857124)

In [58]:
Scoef=linear_svc.coef_

In [59]:
Scoef.shape

(3, 857124)

In [61]:
linear_svc_2= svm.SVC(kernel='linear',C=0.1).fit(X_train,ytrain)

In [62]:
linear_svc_2.score(X_test,ytest)

0.98148148148148151

In [63]:
linear_svc.score(X_test,ytest)

0.98148148148148151